In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LinearRegression
import pickle

# Creating and saving a model

### Diabetes dataset
Ten baseline variables, age, sex, body mass index, average blood pressure, and six blood serum measurements were obtained for each of n = 442 diabetes patients, as well as the response of interest, a quantitative measure of disease progression one year after baseline.

In [2]:
X, y = datasets.load_diabetes(return_X_y = True, as_frame=True)

In [3]:
X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [4]:
y[:5]

0    151.0
1     75.0
2    141.0
3    206.0
4    135.0
Name: target, dtype: float64

### Create the model

In [5]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

### Test the prediction
Model expects 10 features as input, and returns a likelihood value

In [6]:
expected_input = [[0.038076, 0.050680, 0.061696,
             0.021872, -0.044223, -0.034821,
             -0.043401, -0.002592, 0.019908,
             -0.017646]]

expected_result = model.predict(expected_input)
print(expected_result[0])

206.11608996544487


### Exporting the model

Use `pickle.dump()` to export the model

In [7]:
# save the model to disk
filename = 'finalized_model.sav'
with open(filename, 'wb') as file:
    pickle.dump(model, file)

# Loading the model

### Load the model
Use `pickle.load()` to load the model

In [8]:
filename = 'finalized_model.sav'
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

### Verify that it works

In [9]:
x = [[0.038076, 0.050680, 0.061696,
      0.021872, -0.044223, -0.034821,
      -0.043401, -0.002592, 0.019908,
      -0.017646]]

In [10]:
loaded_model.predict(expected_input)[0]

206.11608996544487

# Create a flask server
In this section, I will use **Visual Studio Code** IDE to demonstrate. You may use your IDE of choice like Atom or Sublime instead.

For terminal commands, I will be using **Git Bash** for Windows or **Terminal** for Mac. It is important that we are not using conda for this. More on it later.

### Setup directories

1. In this **api-lesson** folder, Create a new folder called **deployment**
2. Open the `deployment` folder with **Visual Studio Code**
3. Copy `finalized_model.sav` into the **deployment** folder


### Setup virtualenv
4. Open the terminal and change directory to the 'api-lesson' folder:
    ```bash
    cd path/to/api-lesson

    ```

5. Install `virtualenv` with pip:
    ```bash
    pip install virtualenv

    ```
    
6. Create a new virtual environment called `myenv`:
    ```bash
    virtualenv -p python3 myenv

    ```
    - You will now notice a new folder created for you called "myenv"

7. Activate the virtual environment we just created:
    ```bash
    source myenv/scripts/activate << for windows
     source myenv/bin/activate << for mac

    ```
8. Install the packages we will use for the flask server:
    ```bash
    pip install flask sklearn

    ```

#### Check Point
You should see `(myenv)` in your terminal
<img src="assets/api_01_myenv.png">

Type `pip list` and you should see `flask` and `sklearn` in the list
<img src="assets/api_02_piplist.png">

### Create a Flask server
9. In the **deployment** folder, create a new python file called `app.py`
10. Open `app.py` in **visual studio code**
11. In `app.py`, Write the following code:

```python
# Loads Flask Library
from flask import Flask
# initialises a flask application
app = Flask(__name__)

# hello world test
@app.route('/hello') # route defines the API's URI: http://localhost:5000/hello
# function immediately after route will be executed
def hello(): 
    # What the flask server will return
    return "hello world"


# runs the app
if __name__ == '__main__':
  app.run(debug=True)
```

Now that we have created the flask server, we need to **set our environment variables** and run it.

12. In the terminal type:

```bash
cd deployment
export FLASK_ENV=development
export FLASK_APP=app.py
flask run --host=0.0.0.0
```

#### Check Point
Open your browser and enter the URL: `http://localhost:5000/hello`. You should see
<img src="assets/api_03_heloworld.png">

**Hooray we just created our first REST API**.
- We made a `GET` request to the flask server with the URL `http://localhost:5000/hello`
- The server responed with `hello world`

### Create an API to predict diabetes

Next we want to be able to predict diabetes by giving the server **X** and getting **y_pred** in return. For this to happen, we will be passing the **X** as parameters in the URL like so
<img src="assets/api_04_predictionapi.png">
The API URI is `http://localhost:5000/api/diabetes` and the parameter is `?x=0,1,2,3,4,5,6,7,8,9`. This will result in the server giving us the prediction of `9604.737495823083`

13. Change the `app.py` code to the following and **save**:

```python
from flask import Flask
app = Flask(__name__)

# handles REST API requests
from flask import request
# loads our model
import pickle

@app.route('/hello')
def hello(): 
    return "hello world"

# load the model
filename = 'finalized_model.sav'
with open(filename, 'rb') as file:
    model = pickle.load(file)

# predicts diabetes
# expected uri: http://localhost:5000/api/diabetes?x=0,1,2,3,4,5,6,7,8,9
@app.route('/api/diabetes')
def diabetes():
    # get the parameter named "x" in string type
    params = request.args.get('x')
    # change "0,1,2,3,4,5,6,7,8,9" to [[0,1,2,3,4,5,6,7,8,9]]
    x = [[float(i) for i in params.split(',')]]
    # use the model to predict
    pred = model.predict(x)[0]
    # sends the prediction back to client
    return str(pred)

# runs the app
if __name__ == '__main__':
  app.run(debug=True)
```

#### Check Point
Open your browser and enter the URL:<br>
`http://localhost:5000/api/diabetes?x=0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646`

You should see
<img src="assets/api_05_predictionapi2.png">

**We have just created an API that will predict the likelyhood of getting diabetes!**

### Create a web page that predicts diabetes

Next we will create a HTML webpage that uses the API we created
<img src="assets/api_06_webpage.png">

14. Create a new folder called `templates` and in the folder, create a file called `index.html`. The file structure should look like this:
<img src="assets/api_07_filestructure.png">

15. Open `index.html` and add the following code:

```html
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />

    <script
      src="https://code.jquery.com/jquery-3.5.1.min.js"
      integrity="sha256-9/aliU8dGd2tb6OSsuzixeV4y/faTqgFtohetphbbj0="
      crossorigin="anonymous"
    ></script>

    <title>Diabetes Prediction Website</title>
  </head>
  <body>
    <h1>Diabetes Likelihood Calculator</h1>
    <form>
      <label for="age">age:</label><br />
      <input type="text" id="0" name="age" value="0.038076" /><br />

      <label for="sex">sex:</label><br />
      <input type="text" id="1" name="sex" value="0.050680" /><br />

      <label for="bmi">bmi:</label><br />
      <input type="text" id="2" name="bmi" value="0.021872" /><br />

      <label for="bp">bp:</label><br />
      <input type="text" id="3" name="bp" value="0.038076" /><br />

      <label for="s1">s1:</label><br />
      <input type="text" id="4" name="s1" value="-0.044223" /><br />

      <label for="s2">s2:</label><br />
      <input type="text" id="5" name="s2" value="-0.034821" /><br />

      <label for="s1">s3:</label><br />
      <input type="text" id="6" name="s3" value="-0.043401" /><br />

      <label for="s4">s4:</label><br />
      <input type="text" id="7" name="s4" value="-0.002592" /><br />

      <label for="s5">s5:</label><br />
      <input type="text" id="8" name="s5" value="0.019908" /><br />

      <label for="s6">s6:</label><br />
      <input type="text" id="9" name="s6" value="-0.017646" /><br />

      <input type="submit" id="submit" />
    </form>

    <p>Diabetes likelihood is: <span id="result">_</span></p>

    <script language="JavaScript">
      // This is JQUERY scirpt that sends the 10 input values as "x" to the API
      $("#submit").click(function (event) {
        event.preventDefault();
        $.ajax({
          url: "http://localhost:5000/api/diabetes",
          type: "get",
          data: {
            x:
              $("#0").val() +
              "," +
              $("#1").val() +
              "," +
              $("#2").val() +
              "," +
              $("#3").val() +
              "," +
              $("#4").val() +
              "," +
              $("#5").val() +
              "," +
              $("#6").val() +
              "," +
              $("#7").val() +
              "," +
              $("#8").val() +
              "," +
              $("#9").val(),
          },
          success: function (response) {
            $("#result").html(response);
          },
        });
      });
    </script>
  </body>
</html>


```

If you open the `index.html` file now, it should look correct. **BUT if you submit, it will not work yet** because the website has to be served from our flask server first! Our final step will be to serve this html on our flask server.

16. In `app.py` change the code to the following:

```python
from flask import Flask
app = Flask(__name__)
from flask import request
import pickle

# used to serve html files
# files have to be inside "templates" folder
from flask import render_template

@app.route('/hello')
def hello(): 
    return "hello world"

# serve website
@app.route('/')
def home():
    # notice that "./index.html" actually means "templates/index.html"
    # this is because render_template() treats 'templates' folder as the root
    return render_template("./index.html")


filename = 'finalized_model.sav'
with open(filename, 'rb') as file:
    model = pickle.load(file)

@app.route('/api/diabetes')
def diabetes():
    params = request.args.get('x')
    x = [[float(i) for i in params.split(',')]]
    pred = model.predict(x)[0]
    return str(pred)

if __name__ == '__main__':
  app.run(debug=True)
```

#### Final Check Point

Open `http://localhost:5000/` and you should see a fully functioning Diabetes calculator. You have successfully created a web-service API that predicts using your machine learning model and a web application to demonstrate it!

### Create a Requirements file

The last thing we need to do is to create a `requirements.txt` file so that we can easily install the packages used in this project if we want to replicate it.

17. You can terminate the server via your **Git Bash** by typing `CTRL+c`
18. in git bash type: `pip freeze > requirements.txt`
    - you should see a new file created called `requirements.txt`
    - In the future, you can install all packages for this project by typing `pip install -r requirements.txt`
19. To exit `myenv` type in the terminal: `deactivate`


Below is the full `app.py` code for reference:
```python
# Loads Flask Library
from flask import Flask
# initialises a flask application
app = Flask(__name__)

# used to serve html files
# files have to be inside "templates" folder
from flask import render_template

# handles REST API requests
from flask import request

# loads our model
import pickle


# hello world test
@app.route('/hello') # route defines the API's URI: http://localhost:5000/hello
# function immediately after route will be executed
def hello(): 
    # What the flask server will return
    return "hello world"

# serve website
@app.route('/')
def home():
    # notice that "./index.html" actually means "templates/index.html"
    # this is because render_template() treats 'templates' folder as the root
    return render_template("./index.html")


# load the model
filename = 'finalized_model.sav'
with open(filename, 'rb') as file:
    model = pickle.load(file)

# predicts diabetes
# expected uri: http://localhost:5000/api/diabetes?x=0,1,2,3,4,5,6,7,8,9
@app.route('/api/diabetes')
def diabetes():
    # get the parameter named "x"
    params = request.args.get('x')
    # change "0,1,2,3,4,5,6,7,8,9" to [[0,1,2,3,4,5,6,7,8,9]]
    x = [[float(i) for i in params.split(',')]]
    # use the model to predict
    pred = model.predict(x)[0]
    # sends the prediction back to client
    return str(pred)

# runs the app
if __name__ == '__main__':
  app.run(debug=True)
```